In [296]:
import spacy
import numpy as np
import pandas as pd

from scipy import spatial
from langdetect import detect
from sentence_transformers import SentenceTransformer

In [297]:
# Import data
municipality_names = pd.read_csv('../data/list_of_municipalities_italy.csv', 
                                 delimiter=';',
                                 dtype={'LAU_NAME':str})

municipality_names['LAU_NAME'][267] = 'None'
step3 = pd.read_table('../data/interim_matched_data.csv', 
                      delimiter=';',
                      dtype={'LAU_ID':str,'lau_id_step2': str})



<ipython-input-297-e45e1bba6321>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipality_names['LAU_NAME'][267] = 'None'


In [259]:
# Load embeddings
nlp_fr = spacy.load('fr_core_news_lg')
nlp_en = spacy.load('en_core_web_lg')
nlp_it = spacy.load('it_core_news_lg')

In [298]:
model = SentenceTransformer('nickprock/sentence-bert-base-italian-xxl-uncased')

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.15k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/732k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/243k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [300]:
french_embedding = step3.apply(lambda x: model.encode(x['location']), axis=1)
italian_embedding = municipality_names.apply(lambda x: model.encode(x['LAU_NAME']), axis=1)

In [302]:
# Calculate the similarity between the entry in step3 and each of the municipalities
##french_embedding = step3.apply(lambda x: nlp_it(x['location']).vector, axis=1)
##italian_embedding = municipality_names.apply(lambda x: nlp_it(x['LAU_NAME']).vector, axis=1)
distance = np.ones((french_embedding.shape[0], italian_embedding.shape[0])) + 10

print(len(italian_embedding))
print(distance.shape)

8003
(3324, 8003)


In [308]:
# Fill in the distance matrix
for i in range(len(french_embedding)):
    for j in range(len(italian_embedding)):
        distance[i, j] = spatial.distance.cosine(french_embedding[i], italian_embedding[j])
        

In [307]:
# Find the minimum distance and which word that is
spatial.distance.cosine(french_embedding[0], italian_embedding[298])

#municipality_names.iloc[422]

0.6299591958522797

In [309]:
# Compute the minimum value for each row
min_values = np.min(distance, axis=1)
# Compute the indices of the minimum values for each row
min_indices = np.argmin(distance, axis=1)

In [310]:
municipality_names.iloc[min_indices]

,LAU_NAME,LAU_ID
2489,Turi,72047
2489,Turi,72047
2489,Turi,72047
7815,Voghera,18182
5418,Calcio,16044
...,...,...
6161,Borgomale,4024
5902,Chiusaforte,30025
1054,Nicotera,102025
1054,Nicotera,102025


In [295]:
step3

,text,source,class,location,respones,LAU_ID,candidate_match_step2_1,candidate_match_step2_2,candidate_match_step2_3,candidate_match_step2_4,...,candidate_match_step2_7,candidate_match_step2_8,candidate_match_step2_9,candidate_match_step2_10,candidate_match_step2_11,candidate_match_step2_12,candidate_match_step2_13,candidate_match_step2_14,candidate_match_step2_15,lau_id_step2
0,"1. Dalmasso (Henry), imprimerieroyale, à Turin...",Paris1867_Italy_0.jpg,CLASSE 6..— PRUDLITS D'IMPRIMERIE\nET DE LIBRA...,Turin,Turin,NaN,Turin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"2. Negro (AugusteFréderic), à Turin. — Ouvrage...",Paris1867_Italy_0.jpg,CLASSE 6..— PRUDLITS D'IMPRIMERIE\nET DE LIBRA...,Turin,The Italian city mentioned in the given string...,001272,Torino,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001272
2,"3. Pomba (Louis), à Turin.—Publications de la ...",Paris1867_Italy_0.jpg,CLASSE 6..— PRUDLITS D'IMPRIMERIE\nET DE LIBRA...,Turin,Turin (or Torino in Italian) is the city menti...,001272,Torino,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001272
3,"4. Gatti (Joseph), à Voghera. —Spécimens de ty...",Paris1867_Italy_0.jpg,CLASSE 6..— PRUDLITS D'IMPRIMERIE\nET DE LIBRA...,Voghera,The Italian city mentioned in the given string...,018182,Voghera,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,018182
4,"5. Bernardoni (CièreJoseph , àMilan. — Evangel...",Paris1867_Italy_0.jpg,CLASSE 6..— PRUDLITS D'IMPRIMERIE\nET DE LIBRA...,Milan,Milan,NaN,Milan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3319,"14. Cherick (JeanMarie), à BorgoSan Sepoléro (...",Paris1867_Italy_116.jpg,CLASSE 92 — SPÉCIMENS DE COs-\nTUMES POPULAIRE...,BorgoSan,The Italian city mentioned in the given string...,051002,Arezzo,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,051002
3320,4. Municipalité de Cilenza(Foggia). — Costume ...,Paris1867_Italy_116.jpg,CLASSE 92 — SPÉCIMENS DE COs-\nTUMES POPULAIRE...,Valforte,The city mentioned in the given string is Cile...,NaN,Cilenza,Cilento,Cilento,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3321,"3. Brameia (Vincent, à Nicotera(Catanzara. — P...",Paris1867_Italy_116.jpg,CLASSE 92 — SPÉCIMENS DE COs-\nTUMES POPULAIRE...,Nicotera,The Italian city mentioned in the given string...,102025,Nicotera,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102025
3322,"9. Russo (Dominique), à Nicotera(Catanzaro). —...",Paris1867_Italy_116.jpg,CLASSE 92 — SPÉCIMENS DE COs-\nTUMES POPULAIRE...,Nicotera,The Italian city mentioned in the given string...,102025,Nicotera,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102025
